In [1]:
import sys
from pathlib import Path

# Add src directory to Python path
project_root = Path.cwd().parent
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
    print(f"Added {src_path} to Python path")

Added /Users/cooper/Desktop/hydro-forecasting/src to Python path


In [2]:
import polars as pl
from sklearn.pipeline import Pipeline
from hydro_forecasting.data.caravanify_parquet import (
    CaravanifyParquet,
    CaravanifyParquetConfig,
)
from hydro_forecasting.preprocessing.grouped import GroupedPipeline
from hydro_forecasting.preprocessing.normalize import NormalizeTransformer
from hydro_forecasting.preprocessing.standard_scale import StandardScaleTransformer
from hydro_forecasting.experiment_utils.train_model_from_scratch import train_model_from_scratch

---

## Loading the data (as gauge_ids)

In [3]:
config_ca = CaravanifyParquetConfig(
    attributes_dir="/Users/cooper/Desktop/CaravanifyParquet/CA/post_processed/attributes",
    timeseries_dir="/Users/cooper/Desktop/CaravanifyParquet/CA/post_processed/timeseries/csv",
    shapefile_dir="/Users/cooper/Desktop/CAMELS-CH/data/CARAVANIFY/CA/post_processed/shapefiles",
    gauge_id_prefix="CA",
    use_hydroatlas_attributes=True,
    use_caravan_attributes=True,
    use_other_attributes=True,
)

caravan_ca = CaravanifyParquet(config_ca)
basin_ids = caravan_ca.get_all_gauge_ids()[:10]

# basin_ids = [bid for bid in basin_ids if bid != "CA_15030"]

caravan_ca.load_stations(basin_ids)

## Datamodule Configs

In [4]:
region_time_series_base_dirs = {"CA": "/Users/cooper/Desktop/CaravanifyParquet/CA/post_processed/timeseries/csv/CA"}

region_static_attributes_base_dirs = {"CA": "/Users/cooper/Desktop/CaravanifyParquet/CA/post_processed/attributes/CA"}

path_to_preprocessing_output_directory = "/Users/cooper/Desktop/hydro-forecasting/tests/yolo_7"

In [5]:
forcing_features = [
    "snow_depth_water_equivalent_mean",
    "surface_net_solar_radiation_mean",
    "surface_net_thermal_radiation_mean",
    "potential_evaporation_sum_ERA5_LAND",
    "potential_evaporation_sum_FAO_PENMAN_MONTEITH",
    "temperature_2m_mean",
    "temperature_2m_min",
    "temperature_2m_max",
    "total_precipitation_sum",
]

static_features = [
    "p_mean",
    "area",
    "ele_mt_sav",
    "high_prec_dur",
    "frac_snow",
    "high_prec_freq",
    "slp_dg_sav",
    "cly_pc_sav",
    "aridity_ERA5_LAND",
    "aridity_FAO_PM",
]

target = "streamflow"

In [6]:
feature_pipeline = GroupedPipeline(
    Pipeline([("scaler", StandardScaleTransformer()), ("normalizer", NormalizeTransformer())]),
    columns=forcing_features,
    group_identifier="gauge_id",
)

target_pipeline = GroupedPipeline(
    Pipeline([("scaler", StandardScaleTransformer()), ("normalizer", NormalizeTransformer())]),
    columns=["streamflow"],
    group_identifier="gauge_id",
)

static_pipeline = Pipeline([("scaler", StandardScaleTransformer())])

preprocessing_config = {
    "features": {"pipeline": feature_pipeline},
    "target": {"pipeline": target_pipeline},
    "static_features": {"pipeline": static_pipeline, "columns": static_features},
}

In [7]:
datamodule_config = {
    "region_time_series_base_dirs": region_time_series_base_dirs,
    "region_static_attributes_base_dirs": region_static_attributes_base_dirs,
    "path_to_preprocessing_output_directory": path_to_preprocessing_output_directory,
    "group_identifier": "gauge_id",
    "batch_size": 2048,
    "forcing_features": forcing_features,
    "static_features": static_features,
    "target": target,
    "num_workers": 4,
    "min_train_years": 5,
    "train_prop": 0.5,
    "val_prop": 0.25,
    "test_prop": 0.25,
    "max_imputation_gap_size": 5,
    "chunk_size": 10,
    "is_autoregressive": True,
    "preprocessing_configs": preprocessing_config,
}

## Training Configs

In [8]:
training_config = {
    "max_epochs": 5,
    "accelerator": "mps",
    "devices": 1,
    "precision": 16,
    "early_stopping_patience": 10,
    "reload_dataloaders_every_n_epochs": 1,
}

## Remaining Configs

In [9]:
output_dir = "/Users/cooper/Desktop/hydro-forecasting/tests/yolo_7"
model_types = ["tide", "ealstm"]
yaml_paths = [
    "/Users/cooper/Desktop/hydro-forecasting/experiments/TiDE_on_low_medium/yaml_files/tide.yaml",
    "/Users/cooper/Desktop/hydro-forecasting/experiments/TiDE_on_low_medium/yaml_files/ealstm.yaml",
]
experiment_name = "first_test"
num_runs = 2
override_previous_attempts = False

## Training the models from scratch

In [10]:
train_results = train_model_from_scratch(
    gauge_ids=basin_ids,
    datamodule_config=datamodule_config,
    training_config=training_config,
    output_dir=output_dir,
    model_types=model_types,
    yaml_paths=yaml_paths,
    experiment_name=experiment_name,
    num_runs=num_runs,
    override_previous_attempts=override_previous_attempts,
)

INFO:hydro_forecasting.experiment_utils.train_model_from_scratch:Starting training for experiment 'first_test'
INFO:hydro_forecasting.experiment_utils.train_model_from_scratch:Output directory: /Users/cooper/Desktop/hydro-forecasting/tests/yolo_7/first_test
INFO:hydro_forecasting.experiment_utils.train_model_from_scratch:Models to train: tide, ealstm
INFO:hydro_forecasting.experiment_utils.train_model_from_scratch:Number of runs per model: 2
INFO:hydro_forecasting.experiment_utils.train_model_from_scratch:Processing model (1/2): tide
INFO:hydro_forecasting.data.in_memory_datamodule:Starting data preparation...
INFO:hydro_forecasting.data.in_memory_datamodule:Generated Run UUID for current config: 00c2a4eb-e67c-54d7-a4fe-2f35fe967bad
INFO:hydro_forecasting.data.in_memory_datamodule:Checking for existing processed data at: /Users/cooper/Desktop/hydro-forecasting/tests/yolo_7/00c2a4eb-e67c-54d7-a4fe-2f35fe967bad
INFO:hydro_forecasting.data.in_memory_datamodule:No reusable data found or re

INFO: Processed 10 basins, 9 passed quality checks


INFO:hydro_forecasting.data.preprocessing:--- Finished Batch 1 ---
INFO:hydro_forecasting.data.preprocessing:Finished processing all time series batches. Attempted 10 basins.
INFO:hydro_forecasting.data.preprocessing:Fitted time series pipelines saved to /Users/cooper/Desktop/hydro-forecasting/tests/yolo_7/00c2a4eb-e67c-54d7-a4fe-2f35fe967bad/fitted_time_series_pipelines.joblib
INFO:hydro_forecasting.data.preprocessing:Summary quality report saved to /Users/cooper/Desktop/hydro-forecasting/tests/yolo_7/00c2a4eb-e67c-54d7-a4fe-2f35fe967bad/quality_summary.json
INFO:hydro_forecasting.data.preprocessing:SUCCESS: Preprocessing completed successfully. Output at /Users/cooper/Desktop/hydro-forecasting/tests/yolo_7/00c2a4eb-e67c-54d7-a4fe-2f35fe967bad
INFO:hydro_forecasting.data.in_memory_datamodule:Hydro processor completed successfully.
INFO:hydro_forecasting.data.in_memory_datamodule:Successfully loaded 3 categories of fitted pipelines.
INFO:hydro_forecasting.data.in_memory_datamodule:9 ba

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

INFO:hydro_forecasting.data.in_memory_datamodule:Epoch 0: Val Dataloader using cached validation data with 14266 samples from 9 basins.
/Users/cooper/Desktop/hydro-forecasting/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: The combination of `DataLoader(`pin_memory=True`, `persistent_workers=True`) and `Trainer(reload_dataloaders_every_n_epochs > 0)` can lead to instability due to limitations in PyTorch (https://github.com/pytorch/pytorch/issues/91252). We recommend setting `pin_memory=False` in this case.
/Users/cooper/Desktop/hydro-forecasting/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


...

INFO:hydro_forecasting.data.in_memory_datamodule:Epoch 4: Val Dataloader using cached validation data with 14248 samples from 9 basins.


Epoch 4: 100%|██████████| 15/15 [00:06<00:00,  2.17it/s, v_num=pt_0]

Metric val_loss improved by 0.032 >= min_delta = 0.0. New best score: 0.067
`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 15/15 [00:07<00:00,  2.14it/s, v_num=pt_0]


INFO:hydro_forecasting.experiment_utils.train_model_from_scratch:Run 1 completed with best val_loss: 0.06719012558460236
INFO:hydro_forecasting.experiment_utils.checkpoint_manager:Updated overall_best_model_info.txt at /Users/cooper/Desktop/hydro-forecasting/tests/yolo_7/first_test/checkpoints/ealstm to point to: run_0/attempt_0/ealstm-run0-attempt_0-epoch=04-val_loss=0.0603.ckpt
INFO:hydro_forecasting.experiment_utils.train_model_from_scratch:Overall best model for ealstm: run_0/attempt_0/ealstm-run0-attempt_0-epoch=04-val_loss=0.0603.ckpt
INFO:hydro_forecasting.experiment_utils.train_model_from_scratch:Best metrics: {'val_loss': 0.06027819961309433, 'run_index': 0, 'seed': 42}
